### 1.Inspect dataset floder

In [754]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### 2.Load dataset

In [755]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
# Check Nan on train-set and test-set
# print("train_data \n ",train_data.isnull().sum())
# print("__________________________________")
print(train_data.info())
train_data.head()

In [756]:
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
# print("test_data \n ",test_data.isnull().sum())
# print("__________________________________")
print(test_data.info())
test_data.head()

In [757]:
gender_sub = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')
gender_sub.head()

### 3.EDA (Explore Data Analysis)

#### Variable Description

**Categorical Features (Qualitative):**

*Nominal*

- Sex: Passenger's gender (female = 0,male = 1)
- Embarked: Port of Embarkation (C = Cherbourg, Q = Queenstown, S = Southampton)
- Survived: Passenger survive(1) or died(0)
- Name: Name of passenger

*Ordinal*

- Pclass: Ticket class

**Quantitative :**

*Discrete*

- PassengerID: Passenger's identification
- SibSp: Number of siblings/spouses
- Fare: Price paid for ticket
- Parch: Number of parents or child on board along 

*Continous*

- Age: Age of passenger at the time

**Mixed Data Types**

*Alphanumerical*

- Cabin: Cabin number
- Ticket - Ticket number

#### Predicting features

- Survived: Passenger survive (1) or died (0)

#### Interesting features

- Sex: Passenger's gender (female = 0,male = 1)
- Embarked: Port of Embarkation (C = Cherbourg, Q = Queenstown, S = Southampton)
- Pclass: Ticket class
- SibSp: Number of siblings/spouses
- Fare: Price paid for ticket
- Parch: Number of parents or child on board along
- Age: Age of passenger at the time

#### Some observations:

- The correllation matrix
- Embarked Survived
- Sex Survived
- Pclass Survived
- SibSp Survived
- Parch Survived
- Age Survived
- Higher Fare = higher survival rate ?

In [758]:
# make descript about number column
train_data.describe()

In [759]:
test_data.describe()

- Correlation matrix

In [760]:
fig,ax = plt.subplots(figsize = (15,8))
sns.heatmap(train_data.corr(),fmt = '.2f',annot = True)
ax.set_title('Correlation Matrix',fontsize = 20)

- Embark-Survived
- Sex-Survived
- Pclass Survived
- SibSp Survived
- Parch Survived

In [761]:
fig,ax = plt.subplots(1,5,figsize = (15,5),sharey = True)
fig.suptitle('Pclass-SibSp-Parch-Survived')
sns.countplot(ax  =ax[0],x = 'Embarked', hue = 'Survived', data = train_data)
ax[0].set_title('Embark-Survived')
sns.countplot(ax  =ax[1],x = 'Sex', hue = 'Survived', data = train_data)
ax[1].set_title('Sex-Survived')
sns.countplot(ax  =ax[2],x = 'Pclass', hue = 'Survived', data = train_data)
ax[2].set_title('Pclass-Survived')
sns.countplot(ax  =ax[3],x = 'SibSp', hue = 'Survived', data = train_data)
ax[3].set_title('SibSp-Survived')
sns.countplot(ax  =ax[4],x = 'Parch', hue = 'Survived', data = train_data)
ax[4].set_title('Parch-Survived')
plt.tight_layout()

- Age Survived

In [762]:
fig,ax = plt.subplots(1,figsize = (10,7))
sns.distplot(train_data['Age'].dropna().values,kde = True)

In [763]:
train_data['Age'] = train_data['Age'].fillna(value = train_data.Age.mean())
test_data['Age'] = test_data['Age'].fillna(value = test_data.Age.mean())

In [764]:
train_data['Age_Category'] = pd.cut(train_data['Age'],bins = [0,16,40,60,90])
plt.subplots(figsize = (14,10))
sns.countplot(x= 'Age_Category',hue = 'Survived',data= train_data)
plt.show()

Higher fare = higher survival rate?

In [765]:
train_data[train_data['Fare'] > train_data['Fare'].mean()].Survived.value_counts(normalize=  True)

In [766]:
test_data['Fare'] = test_data['Fare'].fillna(test_data['Fare'].mean())

In [767]:
fig,ax = plt.subplots(figsize = (10,7))
sns.distplot(train_data['Fare'].values,kde =True)

In [768]:
plt.subplots(figsize = (14,10))
train_data['Fare_Category'] = pd.cut(train_data['Fare'],bins= [0,40,100,600])
sns.countplot(x = 'Fare_Category',hue = 'Survived',data= train_data)
plt.show()

### 4.Feature engineering

- Feature columns: Embarked, Sex, Pclass, SibSp, Parch, Age, Fare
- Target column: Survived

In [769]:
train_data.head()

Delete useless columns

In [786]:
# copy dataframe 
xy_train = train_data.copy()
xy_test = test_data.copy()

# PassengerId,Name,Ticket,Cabin
del_features = ['PassengerId','Name','Ticket','Cabin']
xy_train = xy_train.drop(columns = del_features)
xy_test = xy_test.drop(columns= del_features)

In [787]:
xy_train = xy_train.dropna(how = 'any')
xy_train.isnull().sum()

In [788]:
xy_test.isnull().sum()

In [789]:
xy_train['Sex'].replace('male',1,inplace =True)
xy_train['Sex'].replace('female',0,inplace =True)
xy_test['Sex'].replace('male',1,inplace =True)
xy_test['Sex'].replace('female',0,inplace =True)

In [790]:
# 0,16,40,60,90
xy_train.loc[xy_train['Age']<=16,'Age'] = 0
xy_train.loc[(xy_train['Age']>16)&(xy_train['Age']<=40),'Age'] = 1
xy_train.loc[(xy_train['Age']>40)&(xy_train['Age']<=60),'Age'] = 2
xy_train.loc[(xy_train['Age']>60)&(xy_train['Age']<=90),'Age'] = 3
xy_train.loc[xy_train['Age']>90,'Age'] = 4
xy_train.isnull().sum()

In [791]:
# 0,16,40,60,90
xy_test.loc[xy_test['Age']<=16,'Age'] = 0
xy_test.loc[(xy_test['Age']>16)&(xy_test['Age']<=40),'Age'] = 1
xy_test.loc[(xy_test['Age']>40)&(xy_test['Age']<=60),'Age'] = 2
xy_test.loc[(xy_test['Age']>60)&(xy_test['Age']<=90),'Age'] = 3
xy_test.loc[xy_test['Age']>90,'Age'] = 4
xy_test.isnull().sum()

In [792]:
xy_train['Age'].value_counts()

In [793]:
xy_test['Age'].value_counts()

Create a column "Family"which will store the number of family members who travel together (you need to add Parch and SibSp)

In [794]:
xy_train['Family'] = xy_train['SibSp'] + xy_train['Parch'] + 1
xy_test['Family'] = xy_test['SibSp'] + xy_test['Parch'] + 1
xy_train['Alone'] = 0
xy_test['Alone'] = 0
xy_train.loc[xy_train['Family']==1,'Alone'] = 1
xy_test.loc[xy_test['Family']==1,'Alone'] = 1

Fare_Category

In [795]:
pd.qcut(xy_train['Fare'],4)

In [796]:
xy_train.loc[xy_train['Fare']<=7.925,'Fare'] = 0
xy_train.loc[(xy_train['Fare']>7.925)&(xy_train['Fare']<=14.5),'Fare'] = 1
xy_train.loc[(xy_train['Fare']>14.5)&(xy_train['Fare']<=31.275),'Fare'] = 2
xy_train.loc[xy_train['Fare']>31.275,'Fare'] = 3

In [797]:
xy_test.loc[xy_test['Fare']<=7.925,'Fare'] = 0
xy_test.loc[(xy_test['Fare']>7.925)&(xy_test['Fare']<=14.5),'Fare'] = 1
xy_test.loc[(xy_test['Fare']>14.5)&(xy_test['Fare']<=31.275),'Fare'] = 2
xy_test.loc[xy_test['Fare']>31.275,'Fare'] = 3

In [798]:
xy_test.Fare.value_counts()

Replace in the Embarked column from 'S' to 0, 'C' to 1, 'Q' to 2

In [799]:
test_data['Embarked'].value_counts()

In [800]:
train_data['Embarked'].value_counts()

In [801]:
xy_train['Embarked'] = xy_train['Embarked'].map({'S':0,'C':1,'Q':2}).astype(int)
xy_test['Embarked'] = xy_test['Embarked'].map({'S':0,'C':1,'Q':2}).astype(int)

In [802]:
xy_train.head()

In [803]:
# Get label
y = xy_train['Survived']

In [804]:
del_features = ['SibSp','Parch','Age_Category','Fare_Category']
xy_train = xy_train.drop(columns = del_features)
xy_train.head()

In [805]:
fig,ax = plt.subplots(figsize = (15,8))
sns.heatmap(xy_train.corr(),fmt = '.2f',annot = True)
ax.set_title('Correlation Matrix',fontsize = 20)

In [806]:
del xy_train['Survived']
xy_train.head()

In [807]:
xy_test.head()

In [808]:
del_features = ['SibSp','Parch']
xy_test = xy_test.drop(columns = del_features)
xy_test.head()

### 5.Train model

RandomForest

In [826]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC # SVM
from sklearn.linear_model import LogisticRegression #Logestic Regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [810]:
X_train,X_valid,y_train,y_valid = train_test_split(xy_train,y,test_size = 0.2, random_state = 1)
print(f'X_train.shape {X_train.shape}')
print(f'X_valid.shape {X_valid.shape}')
print(f'y_train.shape {y_train.shape}')
print(f'y_valid.shape {y_valid.shape}')
X_train.head()

In [827]:
# train the model
model_1 = RandomForestClassifier()#RandomForestClassifier(n_estimators = 100,max_depth = 15,random_state = 1)
model_1.fit(X_train,y_train)

In [839]:
# validation
y_pred_1 = model_1.predict(X_valid)
acc_1= np.sum(y_pred_1 == y_valid)/len(y_valid) * 100
print(f'Accuracy of prediction: {acc_1}')

In [830]:
cm = confusion_matrix(y_true=y_valid, y_pred = y_pred_1)
rp = classification_report(y_pred = y_pred_1, y_true = y_valid)
print(cm)
print(rp)

In [840]:
# train the model
model_2 = SVC(kernel='rbf', gamma=100, C=0.5)
model_2.fit(X_train,y_train)
# validation
y_pred_2 = model_2.predict(X_valid)
acc_2= np.sum(y_pred_2 == y_valid)/len(y_valid) * 100
print(f'Accuracy of prediction: {acc_2}')

In [841]:
# train the model
model_3 = LogisticRegression()
model_3.fit(X_train,y_train)
# validation
y_pred_3 = model_3.predict(X_valid)
acc_3 = np.sum(y_pred_3 == y_valid)/len(y_valid) * 100
print(f'Accuracy of prediction: {acc_3}')

In [849]:
models = pd.DataFrame({
    'Model':['Random Forest','SVM','Logistic Regression'],
    'Score':[acc_1,acc_2,acc_3]
})
models.plot.bar(x = 'Model',y= 'Score',rot = 45)

### 6.Prediction

In [850]:
y_test = model_1.predict(xy_test)
y_test

### 7.Submission

In [851]:
output = pd.DataFrame({'PassengerId':test_data.PassengerId,'Survived':y_test})
output.to_csv('submission.csv',index = False)
print("Your submission was sucessfully saved!")

### References
https://www.kaggle.com/competitions/titanic/overview

https://www.kaggle.com/code/alexisbcook/titanic-tutorial

https://www.kaggle.com/code/mk98kr/eda-and-prediction-titanic-dataset

https://www.kaggle.com/code/debajyotidas1/titanic-advanced-eda-random-forest-tuning

https://www.kaggle.com/code/nkitgupta/advance-data-preprocessing